#### Imports


In [1]:
import pandas as pd
import torch
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from datasets import Dataset
from huggingface_hub import login
import logging
from transformers import (
    TrainingArguments,
    Trainer,
    AutoModelForSequenceClassification,
    # tokenizers
    AutoTokenizer,
    DebertaV2Tokenizer,
    DistilBertTokenizer,
    BertTokenizer,
    RobertaTokenizer,
    ElectraTokenizer,
    AlbertTokenizer,
    XLNetTokenizer,
    MobileBertTokenizer,
    # models
    DebertaV2ForSequenceClassification,
    DistilBertForSequenceClassification,
    BertForSequenceClassification,
    RobertaForSequenceClassification,
    ElectraForSequenceClassification,
    AlbertForSequenceClassification,
    XLNetForSequenceClassification,
    MobileBertForSequenceClassification,
)
from torch.nn import CrossEntropyLoss
# evaluation metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from collections import Counter

import transformers
print(transformers.__version__)
print(transformers.TrainingArguments)

# Cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

4.53.2
<class 'transformers.training_args.TrainingArguments'>


In [2]:
logging.basicConfig(filename='classification_nvidia.log', level=logging.INFO)
logging.info(f"Running on device: {device}")

In [3]:
# setting huggingface token
login(token=os.getenv("HUGGINGFACE_TOKEN"))

os.environ["HF_HOME"] = "huggingface_cache" 
os.environ["TRANSFORMERS_CACHE"] = "huggingface_cache"
os.environ["HUGGINGFACE_HUB_CACHE"] = "huggingface_cache"

print("HF_HOME:", os.getenv("HF_HOME"))
print("TRANSFORMERS_CACHE:", os.getenv("TRANSFORMERS_CACHE"))
print("HUGGINGFACE_HUB_CACHE:", os.getenv("HUGGINGFACE_HUB_CACHE"))

logging.info(f"HF_HOME: {os.getenv('HF_HOME')}")
logging.info(f"TRANSFORMERS_CACHE: {os.getenv('TRANSFORMERS_CACHE')}")
logging.info(f"HUGGINGFACE_HUB_CACHE: {os.getenv('HUGGINGFACE_HUB_CACHE')}")

transformers.utils.hub.TRANSFORMERS_CACHE = "huggingface_cache"

HF_HOME: huggingface_cache
TRANSFORMERS_CACHE: huggingface_cache
HUGGINGFACE_HUB_CACHE: huggingface_cache


### LOADING SQLITE DB WITH RECORDS



In [4]:
import sqlite3
import json
import pandas as pd

DB_FILE = "chunks.db"
OUTPUT_FILE = "exported_chunks.jsonl"

# Connect to the database
conn = sqlite3.connect(DB_FILE)
cur = conn.cursor()

# Query all data from chunks table
cur.execute("SELECT text, label FROM chunks")
rows = cur.fetchall()

# Write to JSONL
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    for text, label in rows:
        obj = {"text": text}
        if label is not None:
            obj["label"] = label
        f.write(json.dumps(obj) + "\n")

conn.close()

print("Data exported to JSONL file.")


Data exported to JSONL file.


In [5]:
# Load the labeled chunks
with open("exported_chunks.jsonl", "r", encoding="utf-8") as f:
    labeled_chunks = [json.loads(line) for line in f]

data = pd.DataFrame(labeled_chunks)
labeled_count = data['label'].value_counts().to_dict()

# Get the first 9000 rows
data = data.head(9000)

# Remove rows with label == 11
data = data[data['label'] != 11]

# Print labeled count after removing label 11
labeled_count = data['label'].value_counts().to_dict()
print(f"Labeled chunks after removing label 11: {labeled_count}")

# Remove rows where label == 1 and text length < 100
data = data[~((data['label'] == 1) & (data['text'].str.len() < 100))]

# Print final labeled count
labeled_count = data['label'].value_counts().to_dict()
print(f"Final labeled chunks: {labeled_count}")


Labeled chunks after removing label 11: {1: 8199, 0: 800}
Final labeled chunks: {1: 5384, 0: 800}


##### Spliting data


In [6]:
# Loading the data
data['label'] = data['label'].astype(int)

# Train-Test Split using stratified sampling
train_df, test_df = train_test_split(data, test_size=0.2, stratify=data['label'], random_state=42)

# since there is a class imbalance, we will compute class weights
# to handle this in the loss function
labels = train_df["label"].values
# Compute class weights
classes = np.unique(labels)
weights = compute_class_weight(class_weight="balanced",
                            classes=classes,
                            y=labels)
class_weights = torch.tensor(weights, dtype=torch.float, device=device)
print("Class weights:", class_weights)

# Convert ing the DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

/home/spark/anaconda3/envs/lmforge/lib/python3.10/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  warnings.warn(


Class weights: tensor([3.8648, 0.5743], device='cuda:0')


In [7]:

# ---- Tuning parameters ----
CONFIG = {
    "epochs": 2,
    "batch_size": 16,
    "batch_size_larger": 8, # Batch size for larger models
    "max_length": [128, 256, 512], # Max length of input sequences
    "learning_rate": 5e-5, # Learning rate for the optimizer
    "learning_rate_larger": 3e-5, # Learning rate for larger models
    "weight_decay": 0.01, # Weight decay for regularization
    "output_dir": "huggingface_cache/classification_models"
}

# ---- Model configurations ----
MODEL_CONFIGS = {
    "deberta": {
        "tokenizer_class": DebertaV2Tokenizer,
        "pretrained_model_name": "microsoft/deberta-v3-small", # params 55M
        "model_class": DebertaV2ForSequenceClassification
    },
    "distilbert": {
        "tokenizer_class": DistilBertTokenizer,
        "pretrained_model_name": "distilbert-base-uncased", # params 66M
        "model_class": DistilBertForSequenceClassification
    },
    "bert": {
        "tokenizer_class": BertTokenizer,
        "pretrained_model_name": "bert-base-uncased", # params 110M
        "model_class": BertForSequenceClassification
    },
    "roberta": {
        "tokenizer_class": RobertaTokenizer,
        "pretrained_model_name": "roberta-base", # params 125M
        "model_class": RobertaForSequenceClassification
    },
    "electra": {
        "tokenizer_class": ElectraTokenizer,
        "pretrained_model_name": "google/electra-small-discriminator", # params 14M
        "model_class": ElectraForSequenceClassification
    },
    "albert": {
        "tokenizer_class": AlbertTokenizer,
        "pretrained_model_name": "albert-base-v2", # params 11M
        "model_class": AlbertForSequenceClassification
    },
    "xlnet": {
        "tokenizer_class": XLNetTokenizer,
        "pretrained_model_name": "xlnet-base-cased", # params 110M
        "model_class": XLNetForSequenceClassification
    },
    "mobilebert": {
        "tokenizer_class": AutoTokenizer,
        "pretrained_model_name": "google/mobilebert-uncased", # params 25M
        "model_class": MobileBertForSequenceClassification
    },
    "albert-base-v1": {
        "tokenizer_class": AlbertTokenizer,
        "pretrained_model_name": "albert-base-v1", # params 12M
        "model_class": AlbertForSequenceClassification
    },
    "albert-large-v2": {
        "tokenizer_class": AlbertTokenizer,
        "pretrained_model_name": "albert-large-v2", # params 18M
        "model_class": AlbertForSequenceClassification
    },
    "albert-xlarge-v2": {
        "tokenizer_class": AlbertTokenizer,
        "pretrained_model_name": "albert-xlarge-v2", # params 60M
        "model_class": AlbertForSequenceClassification
    },
    "albert-xxlarge-v2": {
        "tokenizer_class": AlbertTokenizer,
        "pretrained_model_name": "albert-xxlarge-v2", # params 235M
        "model_class": AlbertForSequenceClassification
    },
    "bert-large-uncased": {
        "tokenizer_class": BertTokenizer,
        "pretrained_model_name": "bert-large-uncased", # params 340M
        "model_class": BertForSequenceClassification
    }
}


In [8]:

# ---- Metric function ----
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}




In [12]:
# ---- Weighted Trainer ----
class WeightedLossTrainer(Trainer):
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = CrossEntropyLoss(weight=self.class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss



In [13]:

# ---- Function to train and evaluate ----
def train_and_evaluate(model_name, train_dataset, test_dataset, data, max_length):
    print(f"\n===== Training {model_name} =====")

    # Model + tokenizer
    cfg = MODEL_CONFIGS[model_name]
    tokenizer = cfg["tokenizer_class"].from_pretrained(cfg["pretrained_model_name"])
    model = cfg["model_class"].from_pretrained(
        cfg["pretrained_model_name"],
        num_labels=len(data['label'].unique())
    )

    # Tokenization
    def tokenize_fn(batch):
        return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=max_length)

    train_enc = train_dataset.map(tokenize_fn, batched=True)
    test_enc = test_dataset.map(tokenize_fn, batched=True)
    train_enc = train_enc.rename_column("label", "labels")
    test_enc = test_enc.rename_column("label", "labels")
    train_enc.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    test_enc.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

    # Training args
    training_args = TrainingArguments(
        output_dir=f"{CONFIG['output_dir']}/{model_name}",
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=CONFIG["learning_rate"],
        per_device_train_batch_size=CONFIG["batch_size"],
        per_device_eval_batch_size=CONFIG["batch_size"],
        num_train_epochs=CONFIG["epochs"],
        weight_decay=CONFIG["weight_decay"],
        logging_dir=f"./logs_{model_name}",
        report_to="none",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        seed=42
    )

    # Trainer
    trainer = WeightedLossTrainer(
        model=model,
        args=training_args,
        train_dataset=train_enc,
        eval_dataset=test_enc,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        class_weights=class_weights
    )

    # Train + evaluate
    # time to train
    t_train_start = torch.cuda.Event(enable_timing=True)
    t_train_end = torch.cuda.Event(enable_timing=True)
    t_train_start.record()
    trainer.train()
    t_train_end.record()
    torch.cuda.synchronize()
    t_train_time = t_train_start.elapsed_time(t_train_end) / 1000  # convert to seconds
    print(f"Training time for {model_name}: {t_train_time:.2f} seconds")

    # Predictions
    t_pred_start = torch.cuda.Event(enable_timing=True)
    t_pred_end = torch.cuda.Event(enable_timing=True)
    t_pred_start.record()
    preds = trainer.predict(test_enc)
    t_pred_end.record()
    torch.cuda.synchronize()
    t_pred_time = t_pred_start.elapsed_time(t_pred_end) / 1000  # convert to seconds
    print(f"Prediction time for {model_name}: {t_pred_time:.2f} seconds")
    y_true = preds.label_ids
    y_pred = np.argmax(preds.predictions, axis=1)

    print(f"\nClassification Report for {model_name}:")
    print(classification_report(y_true, y_pred, digits=4))

    performance = {
        "model": model_name,
        "max_length": max_length,
        "train_time_sec": t_train_time,
        "pred_time_sec": t_pred_time,
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": precision_recall_fscore_support(y_true, y_pred, average='weighted')[2],
        "precision": precision_recall_fscore_support(y_true, y_pred, average='weighted')[0],
        "recall": precision_recall_fscore_support(y_true, y_pred, average='weighted')[1],
    }
    return performance

In [14]:
for model_name in MODEL_CONFIGS.keys():
    for max_len in CONFIG["max_length"]:
        # skip if this (model, max_len) already exists in CSV
        if os.path.exists("model_performance_nvidia.csv") and f"{model_name},{max_len}," in open("model_performance_nvidia.csv").read():
            print(f"Skipping {model_name} with max_length={max_len} (already in CSV).")
            continue

        print(f"\n--- Training {model_name} with max_length={max_len} ---")
        performance = train_and_evaluate(model_name, train_dataset, test_dataset, data, max_len)
        print(performance)
        # save performance to a csv file
        with open("model_performance_nvidia.csv", "a") as f:
            #headers if file is empty
            if os.stat("model_performance_nvidia.csv").st_size == 0:
                f.write("model,max_length,train_time_sec,pred_time_sec,accuracy,f1,precision,recall\n")
            f.write(",".join([str(performance[k]) for k in performance]) + "\n")



--- Training deberta with max_length=128 ---

===== Training deberta =====


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/4947 [00:00<?, ? examples/s]

Map:   0%|          | 0/1237 [00:00<?, ? examples/s]

/tmp/ipykernel_4062358/3185563332.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedLossTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 